# ODSC London 22 September - Missing Data Tutorial

**Alexandru Agachi**, Empiric Capital

m.a.agachi@gmail.com

**Patric Fulop**, University of Edinburgh

patric.fulop@gmail.com

**Paul Pop**, Co Founder and CEO, Neurolabs

paul@neurolabs.eu

**But Why Missing Data??**

We did not have to do much work with missing data in finance or machine vision. However, missing data became a real problem when dealing with clinical medical data, during our applications of machine learning to neuro oncogenetic data for survival prediction.

https://github.com/patricieni/ODSCLondon2018_MissingData

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import numpy  as np
import matplotlib.pyplot as plt
%matplotlib inline

## Introduction

In most fields, missing data is so common that it is a given. You literally design your studies from the beginning with missing data in mind. The problem is particularly acute for surveys, longitudinal studies, and in clinical medicine.

Overall, handling missing data is so important that Wainer (2010) considers it one of the “six necessary tools” that researchers need to master in order to successfully tackle problems in their fields in the 21st century.

**Aim of this tutorial**
1. To give you an understanding of missing data and the statistical problems it raises
2. To provide you with a comprehensive, A to Z, applied tutorial on handling missing data
3. Why Python pandas AND a**R**rrrrrh? Because Python/Pandas do not have the necessary libraries at this time.
3. GitHub repository

**How it's taught**

1. Concepts and theory
2. Code commentary
3. Customer support

patric.fulop@gmail.com

paul@neurolabs.eu

### Missing data is a very difficult and niche topic. Brace yourself.

### What is missing data? 

**"Values that are not available and that would be meaningful for analysis if they were observed" (Little, 2012)**

When you have missing data for variables that are not of interest in your study, you can consider that you do not have missing data. The first step is hence to reduce your dataset to the relevant variables. Only then you can start looking at missing data.

In [2]:
# Use Horse Dataset, with 30% missing values

# Description http://archive.ics.uci.edu/ml/datasets/horse+colic

# This is the path where the file is saved in our system
path = "data/horse.csv"

# We load the data into a pandas dataframe, and tell Pandas that the data is separated by commas
horse_df = pd.read_csv(path, delimiter = ',')

In [3]:
# Let's check the length of the dataset

print(len(horse_df))

299


In [4]:
# Let's look at the first row to get an idea of what the data looks like

horse_df.head(1)

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no


In [5]:
# Can observe already from count that there's 58 values missing for respiratory_rate
horse_df.describe()

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
count,2.990000e+02,239.000000,275.000000,241.000000,53.000000,270.000000,266.000000,101.000000,299.000000,299.000000,299.000000
mean,1.087733e+06,38.168619,72.000000,30.460581,4.707547,46.307407,24.274436,3.039604,3659.709030,90.528428,7.387960
std,1.532032e+06,0.733744,28.646219,17.666102,1.982311,10.436743,27.364194,1.967947,5408.472421,650.637139,127.749768
min,5.184760e+05,35.400000,30.000000,8.000000,1.000000,23.000000,3.300000,0.100000,0.000000,0.000000,0.000000
25%,5.289040e+05,37.800000,48.000000,18.000000,3.000000,38.000000,6.500000,2.000000,2111.500000,0.000000,0.000000
50%,5.303010e+05,38.200000,64.000000,25.000000,5.000000,45.000000,7.500000,2.300000,2322.000000,0.000000,0.000000
75%,5.347360e+05,38.500000,88.000000,36.000000,6.500000,52.000000,56.750000,3.900000,3209.000000,0.000000,0.000000
max,5.305629e+06,40.800000,184.000000,96.000000,7.500000,75.000000,89.000000,10.100000,41110.000000,7111.000000,2209.000000


### We picked up missing data (count row). Now what do we do?

**Assessing how bad the situation is**

In [6]:
# let's actually compute the amount of missing data per variable in our dataset
# pandas's isnull() function selects the null values in our dataset, and sum() adds them up per variable.

horse_df.isnull().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp               60
pulse                     24
respiratory_rate          58
temp_of_extremities       56
peripheral_pulse          69
mucous_membrane           47
capillary_refill_time     32
pain                      55
peristalsis               44
abdominal_distention      56
nasogastric_tube         104
nasogastric_reflux       106
nasogastric_reflux_ph    246
rectal_exam_feces        102
abdomen                  118
packed_cell_volume        29
total_protein             33
abdomo_appearance        165
abdomo_protein           198
outcome                    0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

**Visualization is extremely important, hence our focus on it throughout this tutorial. It is important for you in visualizing/assessing your missing data problem, it is important for communicating this to your audience later on, and it is important for discussing missing data with the domain experts/data collectors.**

In [ ]:
# Use missingno package to visualize missing data
# Very niiiiice

import missingno as msno
msno.matrix(horse_df);

Now let's measure some of these bivariate correlations = how is missingness expressed in our dataset if we take two variables at a time.

R's VIM package allows us to quantify multivariate correlations as well, as we will see in the R notebook for this tutorial.

**Heatmap of nullity correlations, a very important device to understand your pairwise missing data patterns:**

In [ ]:
msno.heatmap(horse_df);

These correlations provide more information about our missing data, than the univariate missing patterns (per each variables) alone.

    1. -1 means that when one variable appears, the other definitely does not appear. 
    2. 0 means that variables have no influence on each other
    3. +1 means that when one variables appears, the other most definitely appears. 

We can observe the highest number is 0.8, **packed_cell_volume** and **total_protein** variables. So we would make the assumption that the protein total and cell volume are part of a similar clinical test, and when one was missed, the other one was too. 

On the other hand there is almost nothing <0, except **respiratory_rate** and **abdomen**, but that doesn't tell us much about the missing values. 

In [ ]:
# dtypes gives us the type of variable. Float and int relate to numbers of course, while object relates to non numbers
horse_df.dtypes.value_counts()

In [ ]:
# Inspect categorical vs numerical, simply for data exploration
numerical = horse_df._get_numeric_data().columns
categorical = list(set(horse_df.columns) - set(numerical))

print(len(categorical)," Categorical Columns are \n",categorical,'\n')
print(len(numerical),"Numerical columns are \n",numerical)

obj=horse_df[categorical]
nonobj=horse_df[numerical]

Below we will use **LabelEncoder** to transform the variables that contain categories represented as words ("strings") to be represented by categories that are numbers. 

We will treat missing values as the number **0**. Although we did do imputation by using fillna, this is just to ease our analysis.

In [ ]:
# Transform them using LabelEncoder without imputing
from sklearn.preprocessing import LabelEncoder

clean_horse_df = horse_df.copy(deep=True)
# Remove nan values in categories by using '0' (as a string)
clean_horse_df[categorical] = clean_horse_df[categorical].fillna('0')
# # Remove nan values in categories by using 0 as an int
# clean_horse_df[numerical] = clean_horse_df[numerical].fillna(0)

clean_horse_df[categorical] = clean_horse_df[categorical].apply(LabelEncoder().fit_transform)

In [ ]:
# we check the different values that the temp_of_extremities variable can take
horse_df.temp_of_extremities.unique()

Below you can observe that for the feature **temp_of_extremities** the word "cool" is encoded by 2 whereas "normal" by 3 and as mentioned previously, missing values "NaN" are 0.

In [ ]:
# first five rows/observations for the variables temp_of_extremities. We see NaN values appear.
horse_df['temp_of_extremities'].head(5)

In [ ]:
# now printing first five rows/observations for the same variable, but from the cleaned/recoded data. NaN is now "0".
clean_horse_df['temp_of_extremities'].head(5)

In [ ]:
# Store the outcome variable in y
y = clean_horse_df['outcome']

### I have missing data. Why do I care?

1. **Increase of bias/Underfitting**: 
    In statistics, bias reflects the extent to which your expected values differ from the true population parameters that you are trying to estimate. Concretely, bias arises in a dataset with missing data whenever for a variable, the missing data differs substantially from the observed data.

    **Longitudinal study** and **dropout** : Let’s take a controlled drug study in medicine. In such a study, one compares the two arms of a study: the treatment arm and the control arm. Bias will depend on the relationship between missingness, treatment, and outcome. It is not uncommon for patients in the treatment arm to drop out due to adverse reactions to the treatment, or due to lack of improvement. In parallel, particularly healthy subjects, or ones who react exceptionally well to the treatment, will have very high completion rates for the study. The former, missing patients, will then become missing data in the final study. Ignoring them would lead to biased estimates of the efficacy of the treatment. We would be estimating how efficient the treatment was for a subsample of the population, the “healthier/better reacting” patients that completed the study, as opposed to the entire target population. 

**2. Reduction of power:**
    In statistics, power, scaled from 0 to 1, is the probability that a hypothesis test correctly rejects the null hypothesis when it is false. This is related to the probability of making a type 2 error i.e. failing to reject the null hypothesis when it is false. The higher the power of your study is, the less likely you are to make a type 2 error.

Statistical power is influenced by two characteristics of your study: 
    1. Sample size 
    2. Variability of the outcomes observed. 
We therefore see that power increases if the sample size increases, or if the variability of the outcomes observed decreases. 

The mechanisms through which missing data directly influences the power of your study:
    1. if you simply delete the observations with missing values, you reduce your effective sample size, and 
    therefore reduce your power
    2. if your missing values are more extreme figures than the observed ones (outliers), which regularly happens in practice, you will underestimate your variability and therefore artificially narrow your confidence interval

**3. Most statistical tools, theoretical and applied are designed for complete datasets**

Almost all standard statistical analysis techniques and their implementations in various softwares were developed for complete datasets, and cannot handle appropriately incomplete ones (Schafer and Graham, 2002).

(C/R) - Classification/Regression


Models| Handle Missing data| Reasoning
---|---|---
Linear Models (C/R) |No| Discriminative model, modelling of output rather than input
Non-linear models: Neural Networks (C/R) |No| Same as above
Decision Trees (C/R) |Yes| Defaults missing data to a node
Random Forest (C/R) |Sometimes|Computationally intensive for growing trees. It also depends on the underlying algorithm, i.e. CART 
Bayesian Models & Generative Models |Yes| They can handle missing data probabilistically since you're modelling the input distribution

Continuing data exploration. We inspect correlations in the dataset and correlation matrix in order to select items for regression.

In [ ]:
train_corr = clean_horse_df.corr()
sns.heatmap(train_corr, vmax=0.8)
corr_values = abs(train_corr['outcome']).sort_values(ascending=False)
print("Correlation of features (X) wrt. outcome (Y) in ascending order:\n", corr_values)

1. Observe how respiratory rates are slightly higher for the ones who died 
2. Observe the peak around 0 for total proteins for the horses who died as well, suggesting total protein is an important measure for the outcome. 
3. Observe how the pulse is slightly lower for horses that lived and almost indistinguishable for horses that were dying or were euthanized..
4. Notice how useful and pretty seaborn is, and if you haven't used it yet, now it's time to add it to your data science toolkit! 

In [ ]:
# Get a feel of the data and correlations. Coding of the variable "outcome": lived = 2, euthanized = 1, died = 0 
g = sns.pairplot(data=clean_horse_df, 
                 vars=['packed_cell_volume','respiratory_rate',
                       'rectal_temp','total_protein','pulse'], 
                 hue='outcome', 
                 height=3);

## We established that, why and how missing data is a problem. Now what?

 If not convinced we'll redo everything up to here. So please be convinced.

## Learn to describe your missing data

## I. Patterns of missing data

1. **Unit vs. item nonresponse** Unit nonresponse means all variables for an observation are missing i.e. a missing patient in a study. Item nonresponse means some variables for some observations are missing.
2. **Univariate** The same observations having missing data across several variables.
3. **Monotone vs. non-monotone** Monotone applies to longitudinal studies, when a patient drops out of a study for example at time t. All values after time t for that observation (patient) will be missing.
4. **Arbitrary** Seemingly random pattern of missing values across the dataset.

![title](img/unit-nonresponse.png)
![title](img/item-nonresponse.png)
![title](img/univariate.png)
![title](img/monotone.png)
![title](img/non-monotone.png)

## Mechanisms of missing data

### How is the data missing?

**1. MCAR: Missing Completely at Random**

If the missingness of the data is *unrelated* to both the observed and the unobserved data, the missing data is said to be missing completely at random (MCAR). In this situation, the missing data is a random subsample of the complete dataset. Discarding the observations with missing data will therefore not bias our estimates. In this specific case, analyzing only the observations with complete data in the study (complete case analysis) would lead to a loss of power/efficiency in the study, but not to a higher bias in the estimates. In practice, MCAR will rarely be the case. 

**2. MAR: Missing at Random** 

If missingness of the data depends only on the observed data and not on the unobserved data, we say that the data is missing at random (MAR). This implies that after taking the observed data into account, there are no systematic differences between items (subjects) with missing data and those without missing data. In practice, this MAR scenario is often the best we can hope for.

**3. MNAR: Missing not at Random**
This occurs when the missingness of the data depends on the values of the missing data themselves. It often occurs for the income variable in surveys for example: both high income earners and low income earners are less likely to report their income (to answer the income question) than are the average income earners. Therefore the fact that those answers will be missing will be correlated to the value of those missing data points themselves. The data in such a study cannot be said to be missing at random.


Most statistical tools require data to be MAR or MCAR. However, in the presence of MNAR, the same statistical tools remain the best thing we have – the study in this case will simply need much more work with the domain experts/data collectors to understand the missingness of the data, and the results will be just much more uncertain/unreliable, meaning they will require extensive sensitivity analyses.

### Now switching to the dark side: aRrrrrh

***But Sensei, if what I've done my whole life was wrong, what can I do now to atone for my sins?***

Impute baby, impute.

## What is imputation? 

What is imputation?

Definition: Imputation amounts to filling in missing values with appropriate estimates for them, and then using standard complete data methods to study the now complete dataset. “From an operational standpoint, imputation solves the missing-data problem at the outset, enabling the analyst to proceed without further hindrance.” (Schafer, 1999)

Objective: It is important to understand from the beginning the objective of imputation. It is not to estimate as accurately as possible any single missing value. The objective is to create a complete dataset that preserves as much as possible the characteristics of the original complete dataset. Intuitively this makes sense because in applied statistics, we aim to infer parameters about a population, based on the estimates computed from our sample. As such we are interested in statistics about aggregates, rather than in the exact value of single observations. As Little and Rubin (2002) state “It is important to note at the outset that usually sample surveys are conducted with the goal of making inferences about population quantities such as means, correlations and regression coefficients, and the values of individual cases in the data set are not the main interest. Thus, the objective of imputation is not to get the best possible predictions of the missing values, but to replace them by plausible values in order to exploit the information in the recorded variables in the incomplete cases for inference about population parameters.” 

Cautionary note: there are many methods to impute missing data. You need to be careful to both use one of the statistically robust methods, and to use a software package that implements it correctly. The theory is very esoteric and there are many permutations possible in implementing it. Choose mainstream multiple imputation methods and mainstream packages, per our recommendations at the end of this tutorial.

### Single imputation
As seen above already actually, in this "technique," missing values are replaced with a single value derived from the non-missing values for each variable, for example the mean or mode of that variable in your dataset.

This is (unfortunately) the most recommended method of imputation, especially in data analysis world. It is also the only method provided by frameworks such as pandas.

What is the problem with single imputation? You are obviously not changing the mean nor mode of your variables since you are using it to fill in the missing values.

Variances and covariances will be severely underestimated (Haitovsky, 1968), for two reasons. First, filling in all missing values with the mean will not account for the variation that would most likely be present in reality between those observed values. You are imputing the mean for every missing value, while the real values would probably vary around the mean value. Second, your ultimately increased sample size will result in smaller standard errors, and these will not accurately reflect the uncertainty actually existing in your dataset due to those missing values. In studying single imputation methods, Pigott (2001) concludes that “under no circumstances does mean imputation produce unbiased results…Bias in the estimation of variances and standard errors are compounded when estimating multivariate parameters such as regression coefficients.”

What about longitudinal studies and the common practice of LOCF (last observation carried forward) and BOCF (baseline observation carried forward)?

They are simply specific cases of single imputation. As such, they do not take into account the information about the missing data contained in the observed data, they lead to biased estimates, and they underestimate the uncertainty contained in the dataset.

Let's see this for ourselves.

### We will pick two variables that have missing values to perform a regression on the outcome

In [ ]:
print("Number of missing values for nasogastric_reflux_ph: %d" %clean_horse_df.nasogastric_reflux_ph.isnull().sum())
print("Number of missing values for nasogastric_reflux_: %d" %(clean_horse_df.nasogastric_reflux==0).sum())

In [ ]:
clean_horse_1 = clean_horse_df.copy(deep=True)
clean_horse_1 = clean_horse_1[['nasogastric_reflux_ph','nasogastric_reflux','outcome']]
clean_horse_1 = clean_horse_1.dropna()

In [ ]:
clean_horse_1.shape

In [ ]:
g = sns.pairplot(data = clean_horse_1, 
                 vars=['nasogastric_reflux_ph','nasogastric_reflux'],
                 hue='outcome',
                 height=3)

Before single imputation, the **nasogastric_reflux_ph** is higher for the dead animals (top left). However, once single imputation has been performed with the mean, we observe a complete overlap between dead and alive (Blue/green) (next figure top left)

In [ ]:
clean_horse_1.nasogastric_reflux_ph.mean()

In [ ]:
clean_horse_1.nasogastric_reflux_ph.mode()

In [ ]:
def get_pairplot(dataframe, fill_na_mode):
    temp_df = dataframe.copy(deep=True)

    temp_df.nasogastric_reflux_ph.fillna(getattr(temp_df.nasogastric_reflux_ph, fill_na_mode)(),inplace=True) 
    temp_df.nasogastric_reflux.fillna(temp_df.nasogastric_reflux.mode(),inplace=True)
    g = sns.pairplot(data = temp_df, 
                     vars=['nasogastric_reflux_ph','nasogastric_reflux'],
                     hue='outcome',
                     height=3)
    return g

In [ ]:
get_pairplot(clean_horse_df, 'mode')

In [ ]:
get_pairplot(clean_horse_df, 'mean')

Let's try imputing **nasogastric_reflux_ph** with the **median**! They are even more confounded.

In [ ]:
get_pairplot(clean_horse_df, 'median')

Imputing with the mode of the data, we observe the distributions are somewhat preserved.

In [ ]:
get_pairplot(clean_horse_df, 'mode')

# Linear estimator analysis
The bias for the linear estimator will increase if the missing data is actually not around the mean. To demonstrate this we will compare the Bias by imputing with the mean and median.

The more different the actual missing data is from the mean, the larger the bias will be. For illustration purposes, we will implement a regression model dependent on two variables: packed_cell_volume and pulse

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
X = clean_horse_df[['packed_cell_volume','pulse','outcome']]

**1. Complete-case analysis: Remove missing values**

Complete case analysis amounts to deleting all observations for which we have missing data. Very commonly done in practice.

If the data is missing completely at random (MCAR), then complete case analysis can be considered (Nakai & Weiming, 2011; Allison, 2001). In this case your reduced dataset will be a random subsample of the full dataset – the parameter estimates therefore will be as unbiased for the reduced dataset as they would have been for the full dataset. 

Allison (2001) also finds that it is more robust than all other non robust methods, such as available case analysis.

Unfortunately, complete case analysis has been shown to produce biased estimates if the data is not missing completely at random (Bell et al, 2013), in addition to the loss of information and power that result from all the observations hence deleted. This is why you should not do this in practice.

"Baseline" regression with non-imputed dataset, removing features that miss

In [ ]:
X_non_imputed = X.dropna()
y_non_imputed = X_non_imputed.outcome
X_non_imputed = X_non_imputed.drop('outcome',axis=1)
# Train the model using the training sets
regr.fit(X_non_imputed, y_non_imputed)

# Make predictions using the testing set
y_pred = regr.predict(X_non_imputed)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_non_imputed, y_pred))
# Explained variance score: 1 is perfect p|rediction
print('Variance score: %.2f' % r2_score(y_non_imputed, y_pred))
val1 = mean_squared_error(y_non_imputed, y_pred) - np.var(y_pred)
print('Bias squared: %.2f' % val1)
r1 = regr.coef_
var1 = np.var(y_pred)

**2. Available Case Analysis: Impute features using their mean**

Available case analysis amounts to taking your incomplete dataset with missing values, and for each variable, using all the observations that do not have missing values for that variable. In practice this means that in the same study you will use different subsamples from your dataset for different variables of interest.

Whaaaaat?

How is this possible? In linear regression we know that a regression can be estimated using only either the sample means and covariance matrix, or the means, standard deviations and correlation matrix (see any introductory statistical book such as Introduction to Statistical Learning). Pairwise deletion aims to take advantage of this insight, computing these statistics using all the cases for which data is available, across our dataset.

We train a linear regression on the imputed data and look at the bias. 

In [ ]:
def run_regression(dataset, fill_na_method, offset=0):
    X_imputed = dataset[['packed_cell_volume','pulse']]
    X_imputed['packed_cell_volume'] = X_imputed['packed_cell_volume'].fillna(
        getattr(X_imputed['packed_cell_volume'], fill_na_method)() - offset)
    X_imputed['pulse'] = X_imputed['pulse'].fillna(
        getattr(X_imputed['pulse'], fill_na_method)() - offset)
    y_imputed = y 
    # Train the model using the training sets
    regr.fit(X_imputed, y_imputed)

    # Make predictions using the testing set
    y_pred = regr.predict(X_imputed)

    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_imputed, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance explained: %.2f' % r2_score(y_imputed, y_pred))
    val2 = mean_squared_error(y_imputed, y_pred) - np.var(y_pred)
    print('Bias squared: %.2f' % val2)

In [ ]:
run_regression(clean_horse_df, 'mean')

**3. Impute using "outliers" or data that is far below the median.**

For illustration purposes we picked median-20

In [ ]:
run_regression(clean_horse_df, 'median', 20)

**4. Imputing with predictions**

i.e. Classifying outcome with ... for example logistic regression!

In [ ]:
# Use only 2 variables and "predict" 20% of the outcomes. 
# For illustration purposes
X_imputed = clean_horse_df[['packed_cell_volume','pulse','outcome']]
X_imputed['packed_cell_volume'] = X_imputed['packed_cell_volume'].fillna(X_imputed['packed_cell_volume'].mean())
X_imputed['pulse'] = X_imputed['pulse'].fillna(X_imputed['pulse'].median())

In [ ]:
# set fixed numpy seed for reproducibility
np.random.seed(123)

msk = np.random.rand(len(X_imputed)) < 0.8

data_train = X_imputed[msk]
data_test = X_imputed[~msk]

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
X_train = data_train[['packed_cell_volume','pulse']]
Y_train = data_train[['outcome']]

X_test = data_test[['packed_cell_volume','pulse']]
Y_test = data_test[['outcome']]
lr_model.fit(X_train,Y_train.values.ravel())

predictions = lr_model.predict(X_test)

In [ ]:
#Inspect predictions vs. original data
predictions - Y_test.values.T

In [ ]:
lr_model.score(X_test,Y_test)

## Now back to R!!!

## Let's finish with a bang not a whisper 

## Rubin's pooling rules for 10 different dataset imputations.

Multiple imputation amounts to generating a desired number of complete datasets, and of then running our analyses on each dataset. However, we need one single result at the end of our analysis. This is what Rubin's rules do: they allow us to combine, in a statistically robust way, the results of our analysis across all our imputed datasets, in order to arrive at one single result.

Do not average the multiply imputed data and then analyze the averages as if it were complete! Why? This would result in incorrect standard errors, confidence intervals, and p values. It would present the same drawbacks as single imputation.

Rubin's rules were first described in his 1987 textbook on handling missing data.

## Is this the Correct version?

In our example below, after multiple imputation, we have 10 datasets that we want to average to find the best 
imputation together with the associated variance.

In [ ]:
mean_estimates = np.zeros((10,1))
data_estimates_var = np.zeros((10,1))

In [ ]:
for i in range(0,10): 
    X_imputed = clean_horse_df[['packed_cell_volume','pulse']]    
    
    # Fill missing values with random numbers but make sure they are positive.
    # We make the assumption of independence between the two variables. 
    imputation_volume = np.abs(X_imputed['packed_cell_volume'].mean() - np.random.randint(0,30))
    X_imputed['packed_cell_volume'] = X_imputed['packed_cell_volume'].fillna(imputation_volume)
    
    imputation_pulse = np.abs(X_imputed['pulse'].median()-np.random.randint(0,30))
    X_imputed['pulse'] = X_imputed['pulse'].fillna(imputation_pulse)
    y_imputed = y
    # The correct estimates 
    # Train the model using the training sets
    regr.fit(X_imputed, y_imputed)

    # Make predictions - these are the estimates
    y_pred = regr.predict(X_imputed)

    weights = regr.coef_
    variance = mean_squared_error(y_imputed, y_pred)
    r2 = r2_score(y_imputed, y_pred)
    #bias = mean_squared_error(y_imputed, y_pred) - np.var(y_pred)
    
    # The coefficients of regression are the estimates. 
    # Could use either
    mean_estimates[i,:] = y_pred.mean()
    #mean_estimates[i,:] = y_pred
    
    # Covariance of 2D dataset
    # Covariance structure - (var(x,x), var(x,y), var(y,x),var(y,y))
    data_estimates_var[i,:] = variance

In [ ]:
# Inspect one mean and covariance Q_l and U_l
print("Means (Estimates): \n",mean_estimates[1])
print("\nVariance of estimates:\n",data_estimates_var[1])

Now onto Rubin's rules

Datasets: $m = 10$  
1. Overall mean for data estimates: $\hat{Q} = \sum_{l=1}^{m}\hat{Q_l}$  
2. Variance-Covariance for data estimates based on covariance of each estimate: $\hat{U} = \sum_{l=1}^{m}\hat{U_l}$

3. Variance in the estimates: $B = \frac{1}{m-1} \sum_{l=1}^{m}(\hat{Q_l}-\hat{Q})^T * (\hat{Q_l}-\hat{Q})$
4. Total variance for real $Q$: $Q-\hat{Q}: T = \hat{U} + B + \frac{B}{m} $  
5. Relative increase in variance due to nonresponse: $r = (1 + \frac{1}{m})*\frac{B}{\hat{U}}$

In [ ]:
#1. Overall means 
Q_hat = mean_estimates.mean(axis=0)
print("Overall Means:",Q_hat)

In [ ]:
#2. Covariance
U_hat = data_estimates_var.mean(axis=0)
print("Overall mean variance in datasets\n",U_hat)

In [ ]:
#3. Variance in estimates - take 1
diff = Q_hat-mean_estimates
B1 = 1/10 * np.dot(diff.T,diff)
print("Take 1: Variance within the estimates\n",B1)

In [ ]:
# Check yourself before you wreck yourself
#3. Variance in estimates - take 2 
B2 = np.var(mean_estimates.T) 
print("Take 1: Variance within the estimates\n",B2)

In [ ]:
(B1 == B2)

In [ ]:
#4. Total variance
T = U_hat + (1+1/10)*B1
print("Total Variance\n",T)

In [ ]:
#5. Relative increase in variance due to nonresponse
r = (1 + 1/10)*B1/U_hat
print("Relative increase in variance due to non-response\n",r)

In [ ]:
# Let's compare to the complete-data mean and covariance
means_complete = r1
var_complete = var1

# We observe they are quite close! 
print("Difference between complete data and imputed data means:\n", means_complete - Q_hat)
print("\nDiference between complete data and imputed data variance:\n", np.abs(var_complete-T))

**Note: Rubin's original rules assumed a normal distribution of the underlying values. Many quantities, such as means, standard deviations, regression coefficients and linear predictors follow this assumption. Other quantities however do not, and need to be tranformed towards normality. Van Buuren (2012) summarized such transformations per the literature:**

<img src="img/Transformations estimators.png">

# Or is this the correct version!? nope.

In [ ]:
# Keep track of all estimates for the artificial 10 datasets we built
# More runs would give better estimates
data_estimates = np.zeros((10,2))
data_estimates_covar = np.zeros((10,2,2))

In [ ]:
for i in range(0,10): 
    X_imputed = clean_horse[['packed_cell_volume','pulse']]    
    
    # Fill missing values with random numbers but make sure they are positive.
    # We make the assumption of independence between the two variables. 
    imputation_volume = np.abs(X_imputed['packed_cell_volume'].mean() - np.random.randint(0,30))
    X_imputed['packed_cell_volume'] = X_imputed['packed_cell_volume'].fillna(imputation_volume)
    
    imputation_pulse = np.abs(X_imputed['pulse'].median()-np.random.randint(0,30))
    X_imputed['pulse'] = X_imputed['pulse'].fillna(imputation_pulse)
    
    # The estimates 
    # Means of each variable
    data_estimates[i,:] = [X_imputed.packed_cell_volume.mean(),X_imputed.pulse.mean()]
    # Covariance of 2D dataset
    # Covariance structure - (var(x,x), var(x,y), var(y,x),var(y,y))
    data_estimates_covar[i,:] = np.cov(X_imputed.T)
    

# Inspect one mean and covariance Q_l and U_l
print("Means: \n",data_estimates[1])
print("\nCovariance:\n",data_estimates_covar[1])

**Now apply Rubin's rules**

In [ ]:
#1. Overall means 
Q_hat = data_estimates.mean(axis=0)
print("Overall Means:",Q_hat)

In [ ]:
#2. Covariance
U_hat = data_estimates_covar.mean(axis=0)
print("Overall mean covariance in datasets\n",U_hat)

In [ ]:
#3. Variance in estimates - take 1
diff = Q_hat-data_estimates
B1 = 1/9 * np.dot(diff.T,diff)
print("Take 1: Covariance within the estimates\n",B1)

In [ ]:
# Check yourself before you wreck yourself
#3. Variance in estimates - take 2 
B2 = np.cov(data_estimates.T) 
print("Take 1: Covariance within the estimates\n",B2)

In [ ]:
#4. Total variance
T = U_hat + (1+1/10)*B1
print("Total Variance\n",T)

In [ ]:
#5. Relative increase in variance due to nonresponse
r = (1 + 1/10)*B1/U_hat
print("Relative increase in variance due to non-response\n",r)

In [ ]:
# Let's compare to the complete-data mean and covariance
means_complete = X_non_imputed.mean(axis=0) 
cov_complete = np.cov(X_non_imputed.T)

# We observe they are quite close! 
print("Difference between complete data and imputed data means:\n", means_complete - Q_hat)
print("\nDiference between complete data and imputed data covariance:\n", cov_complete-T)

_____

### More fun with missing data - Q&A

**How many imputed datasets do you need?**

While Rubin originally suggested that only a couple of imputed datasets would be sufficient (1987), more recent research indicates that significantly more imputations are needed. The appropriate number depends on the proportion of observations with missing values in the dataset. If you have 50% missing values for some variables for example, you will be looking at 40 imputed datasets. A comprehensive study on the desirable number of imputation for different scenarios was published by Graham (2007). He recommends “that one should use m=20, 20, 40, 100, and >100 for true γ= 0.10, 0.30, 0.50, 0.70, and 0.90, respectively.” (m = number of imputed datasets needed and γ = percentage of observations with missing data)

**What variables should you include in your multiple imputation?**

All of them. Rubin (1996) notes that the imputed dataset should include all the variables that will be used for subsequent analyses. This is one of the weaknesses of multiple imputation in practice: the person conducting the multiple imputation and generating the imputed datasets needs to be perfectly calibrated with the researchers conducting the analyses in terms of variables of interest. 
By the same logic, any interaction variables of interest also need to be included. If engineered variables are not included in the multiple imputation dataset, relationships that actually exist may not be found in subsequent analyses, as the imputations would have been generated with the assumption that those variables were independent.

**Including the dependant variable?**

Yes, including that one. This is not only recommended, but essential. For a full reasoning of why, see Allison (2001), Moons, Donders, Stijnen and Harrell (2006) and Graham (2009).

**Should you include any other variables?**

Yes. Collins et al. (2001) showed that you should include in your dataset, before imputation, not only the variables of interest but also all auxiliary variables. These are defined as variables that are not of direct interest in your study themselves, but which are correlated with the variables of interest in your study. Including them in your imputation approach was shown to reduce estimation bias, in particular as due to MNAR data.

**If you have your healthy doubts about the MAR assumption in your dataset, should you still use these multiple imputation methods?**

Yes. They are as good as anything we have, including when the MAR assumption is tenuous (Graham, 2009). It is actually Collins et al. (2001) who showed that MNAR data will not affect in a material way the internal validity of a study by itself. In the medical context, Mallinckrodt et al (2008) and O’Kelly and Ratitch (2014) both reached the same conclusion.

**Should you/can you include categorical variables in your model?**

Yes you should, and yes you can. However, pay attention here as different software implementations of the same algorithms require different preprocessing here (creating dummy variables or not, coding them as factors…).

**Does multiple imputation work for small datasets with many variables?**

Graham & Schafer (1999) showed that “MI performs very well in small samples (as low as N = 50), even with very large multiple regression models (as large as 18 predictors) and even with as much as 50% missing data in the DV.”

**How can you conduct exploratory analysis on a complete dataset given the time that implementing MI requires?**

At this level Allison (2001) offers a simple and elegant solution. When you generate the x imputed datasets that you will use in your multiple imputation procedure, simply generate x+1 instead. Use the extra dataset to conduct your exploratory analysis and evaluation your intended models for analysis.

______

## Handling missing data: roadmap

1. Compute the amount of missing data per variable.
This will tell you how large of a problem you have. In practical terms, it will also be an indication of how much effort you should expense on handling your missing data.

2. Visualize your dataset itself with regards to missing data within it, or generate plots of the missing data across your dataset to check for patterns. If your dataset is longitudinal, check if the missing data is monotone.

3. In complement or replacement to 3, compute in tables the missingness of different variables versus other variables including the response variable.

4. As a follow up to 3, run a logistic regression of missingness for each variable with missing data as the response variable (yes/no), and the other variables as the explanatory variables.
Steps 3 and 4 will allow you to check if any variables are correlated with the missingness in your study, and how.

5. As a result of 3, 4, your investigation of the missing data in this study, and especially your discussion with data collectors and domain experts, decide if you can assume that your data is missing completely at random (MNAR) or at least missing at random (MAR).
If you believe your data is MNAR, you can use Little’s test to confirm this. If your data is MCAR you know that basic methods of handling your missing data such as complete case analysis and available case analysis, will possibly significantly reduce the power of your study, but will not necessarily bias your results.
If you can make the MNAR or MAR assumption, then your missing data is ignorable and you can proceed in confidence with a suitable multiple imputation method and then your intended statistical analyses.
If your data is not missing at random – in this case you will still proceed with the same multiple imputation tools and analyses as above, but you will know that your results may be severely biased. As such, you will not use your results without at the very least complementing them with a suitable sensitivity analysis.

6. Choose and run an appropriate multiple imputation procedure: maximum likelihood, multiple imputation, mice, hot deck or knn.

7. Decide the number m of multiply imputer datasets you will need, per Graham’s guidelines (2007, 2009).

8. Apply your analyses as you would with any other complete dataset.

9. Pool your results following Rubin’s rules.